In [ ]:
import geopandas as gpd
import pystac_client
import pystac
import pandas as pd
import planetary_computer
import rioxarray as rio
import xarray as xr
import stackstac
import matplotlib.pyplot as plt
from shapely import Point, box
from shapely.geometry import shape
import osmnx as ox
import cartopy.crs as ccrs
from xrspatial.multispectral import true_color
import numpy as np
plt.rcParams['font.family'] = 'DejaVu Sans Mono'
from dask.distributed import Client
client=Client()
print(client.dashboard_link)

In [ ]:
### define area of interest 
aoi_geom = shape({"coordinates": [[
    [19.19475654180414,52.727823498032365],
    [19.19475654180414,52.52992257823894],
    [19.553118868416988,52.52992257823894],
    [19.553118868416988,52.727823498032365],
    [19.19475654180414,52.727823498032365]]
                                  ],"type": "Polygon"})

aoi = gpd.GeoDataFrame(geometry=[aoi_geom], crs=4326)

In [ ]:
################# sentinel 2 scenes
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = catalog.search(collections=["sentinel-2-l2a"],
                        intersects=aoi_geom,
                        datetime="2020-01-01/2023-12-31",
                        query={"eo:cloud_cover":{'lt':10}})

s2_items = search.item_collection()
s2_items = pystac.ItemCollection(
    [item for item in s2_items if 
     shape(item.to_dict()['geometry']).contains(aoi_geom)]
)

epsg = set([item.properties['proj:epsg'] for item in s2_items])
print(f'found {len(s2_items)} S2 items\nwith the following epsgs: {epsg}')

epsg_code = list(epsg)[0]
prj = ccrs.epsg(epsg_code)
aoi_prj = gpd.GeoDataFrame(geometry=[aoi_geom],crs=4326).to_crs(prj)['geometry'][0]

s2 = (stackstac.stack(planetary_computer.sign(s2_items),
                      epsg=epsg_code,
                      resolution=20)
      .rio.clip_box(*aoi_prj.bounds))

In [ ]:
band_sel =[1,2,3,4,5,6,7,8,10,11,12]
time_sel = [i for i in range(len(s2['time']))]
_,_,h,w = s2.shape

fig, axs = plt.subplots(figsize=[w/110,h/110],
                        ncols=3,nrows=3,
                        subplot_kw={'projection':prj})

for ax in axs.flat:
    
    t = np.random.choice(time_sel,1)
    bs = np.random.choice(band_sel,3,replace=False)
    
    true_color(
        *s2[t,bs,:,:].squeeze()
            ).plot.imshow(rgb='band',
                      ax=ax)
    ax.annotate(text=pd.to_datetime(s2['time'][t].values[0]).strftime('%d-%m-%Y'),
                xy=(0.98,0.95),
                ha='right',
                xycoords='axes fraction',
                fontsize=12)
    
    ax.annotate(text=' / '.join(s2['band'][bs].values.tolist()),
                xy=(0.98,0.89),
                xycoords='axes fraction',
                ha='right',
                fontsize=12)
    ax.set_axis_off()
    
t = axs.flat[-1].annotate(text='Contains Copernicus Sentinel Data (2023)',
        xy=(0.98,0.03), 
        xycoords='axes fraction', 
        ha='right', 
        fontsize=10,
        c='k')

t.set_bbox({'facecolor':'w','alpha':0.8,'ec':'none'})

t = axs.flat[-3].annotate(text='by:tlohde',
                          xy=(0.02,0.03), 
                          xycoords='axes fraction',
                          ha='left',
                          fontsize=10,
                          c='k')

t.set_bbox({'facecolor':'w','alpha':0.8,'ec':'none'})

x,y = aoi_geom.centroid.coords.xy
x=x[0]
y=y[0]
if y < 0:
    Y = y * -1
    ns = 'S'
else:
    Y = y
    ns = 'N'
    
if x < 0:
    X = x * -1
    ew = 'W'
else:
    X = x
    ew = 'E'

t = axs.flat[0].annotate(text=(f'{Y:.1f}° {ns}, {X:.1f}° {ew}'),
                         xy=(0.02,0.94),
                         xycoords='axes fraction',
                         fontsize=12)

t.set_bbox({'facecolor':'w','alpha':0.8,'ec':'none'})

plt.tight_layout()
plt.subplots_adjust(wspace=0,hspace=0)

In [ ]:
fig.savefig('day21_b.png', dpi=300, bbox_inches='tight')